# Сессия 5

---

Импорт модулей

In [1]:
import nltk
import string
import pickle
import random
import pymorphy2
import regex as re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

Чтение данных

In [2]:
#Как всегда pandas read_csv
df = pd.read_csv("nddata.csv", encoding="utf-8-sig")

Для подачи данных в модель нужно провести векторизацию (на тех же данных что и при обучении)

In [3]:
#обьявление векторизатора
tfidf = TfidfVectorizer()
#обучение векторизатора
tfidf.fit_transform(df['clean_sentence'])

<5367x6545 sparse matrix of type '<class 'numpy.float64'>'
	with 46180 stored elements in Compressed Sparse Row format>

Импорт модели из прошлой сессии

In [4]:
#Для импорта модели буду использовать модуль pickle
model = pickle.load(open("pickle.dat", 'rb'))

Обьявление констант

In [5]:
#Обьявление MorphAnalyzer для использования метода лемматизации
morph = pymorphy2.MorphAnalyzer()
#Создание stop word списка
stop_words = stopwords.words("russian")

Далее помещу в функцию обработку текста для бота

In [6]:
#Функция для обработки nltk и векторизации tfidf
def tokinize_sentence(sentense: str, remove_stop_word: bool = True):
    #Первый шаг это токенизация (будет получен список слов)
    tokens = word_tokenize(sentense, language="russian")
    #второй шаг это удаление пунктуации
    tokens = [i for i in tokens if i not in string.punctuation]
    #Затем удаление стоп слов (если это нам нужно, по дефолту стоит True)
    if remove_stop_word:
        tokens = [i for i in tokens if i not in stop_words]
    #Теперь верну слова в начальнцю форму при помощи MorphAnalyzer
    tokens = [morph.parse(i)[0].normal_form for i in tokens]
    #Удалю все слова короче 4х символов (будет тяжело догадаться что означает "при" или "й")
    tokens = [i for i in tokens if len(i) > 4]
    #обьединю оставшиеся слова в одно предложения для подачи в векторайзер
    column = ' '.join(tokens)
    #поместил векторизированое предложение в переменную X
    X = tfidf.transform([column])
    #И наконец верну предсказанную категорию
    return model.predict(X)[0]

Переменная для вохда в цикл и список с фразами

In [7]:
#Пусть будет пустая, это не так важно потому что она будет перезаписана
inp = ''
#Создам список с фразами для имитации общения с пользователем
phrases = ["Категория вашей жалобы - ", "Полученая жалоба принадлежит к категории - ", "Жалоба будет рассмотена под категорией - ", "Категоря к которой принадлежит ваша жалоба - ", "Категоря относящеяся к вашей жалобе - "]

#### Реализация бота

In [8]:
#Выведу комманды для пользователя перед началом работы бота
print("\nКомманды:\nпредсказание - выдает категорию жалобы по введеному предложению\nинфо - повторяет комманды\nвыход - завершает работу бота\n")
#Цикл бота
while inp != "выход":
    inp = input("Введите комманду - ")
    if inp == "инфо":
        print("\nКомманды:\nпредсказание - выдает категорию жалобы по введеному предложению\nинфо - повторяет комманды\nвыход - завершает работу бота\n")
    if inp == "предсказание":
        text = input("Введите предложение - ")
        #Удалю все специальные символы и цифры с помощью регулярных выражений, а strip удалит лишние пробелы в начале и в конце предложения
        text = re.compile('[^а-яА-Я ]').sub('', text).strip()
        #Использую функцию
        inp2 = tokinize_sentence(text)
        print("\nВаша жалоба отправлена на рассмотрение:\n" + random.choice(phrases) + inp2 + "." + "\n")


Комманды:
предсказание - выдает категорию жалобы по введеному предложению
инфо - повторяет комманды
выход - завершает работу бота

Введите комманду - предсказание
Введите предложение - Добрый, день,, ЧЧЧ у меня вопросы, по ПоВодУ - отопления?

Ваша жалоба отправлена на рассмотрение:
Категоря к которой принадлежит ваша жалоба - Центральное отопление

Введите комманду - выход


Добрый, день,, ЧЧЧ у меня вопросы, по ПоВодУ - отопления?

### Подготовка отчета